# 1 - Deploy RDS Postgres

---
This notebook needs sufficient permission to inspect VPC details, create and modify security groups, and deploy an RDS instance. If you're running this in a Workshop Studio account, copy and paste the credientials for your account below. 

---

In [ ]:
### INSERT YOUR WORKSHOP ACCOUNT CREDENTIALS HERE: 
AWS_ACCESS_KEY_ID="####"
AWS_SECRET_ACCESS_KEY="####"
AWS_SESSION_TOKEN="####"


---
### Deploy dependencies and imports

In [ ]:
!python -V # should be 3.10.x

In [ ]:
!pip install --upgrade pip --quiet
!pip install --upgrade boto3 --quiet
!pip install --upgrade urllib3 --quiet # needed for boto3
!pip install --upgrade psycopg2 --quiet
!pip install --upgrade pgvector --quiet



In [ ]:
import json, boto3
import psycopg2
from pgvector.psycopg2 import register_vector
from requests import get



### Setup globals and boto3 clients
* Note - This code assumed you're running in a Studio Workshop account with a default VPC and Sagemaker Studio setup. Replace the region with your region if not running in us-east-1. 

In [ ]:

REGION = "us-east-1" ### change this as needed 
DBPASS="llama-falcon-laser-narwhal"

AZ = REGION + "a"
SUBNET = None
VPCID = ""

# DB Settings
RDS_POSTGRES_PORT=5432
DBNAME="pgvectorlab"
DBINSTANCE_ID="studiolab-pgvector-instance"
DBINSTANCE_TYPE="db.m6g.large"
DBENGINE="postgres"
DBENGINE_VERSION="15.3"
DBUSERNAME="studiolab"

ip_address = ""


# instance boto3 ec2 client with credentials
ec2_client = boto3.client('ec2',
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                    aws_session_token=AWS_SESSION_TOKEN,
                    region_name=REGION)

rds_client = boto3.client('rds',
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                    aws_session_token=AWS_SESSION_TOKEN,
                    region_name=REGION)

#### Get the public IP of the Sagemaker Studio gateway

In [ ]:
# Get the public IP address of the IGW used by Sagemaker Studio
ip_address = get('https://api.ipify.org').content.decode('utf8')
print('Public IP: {}'.format(ip_address))

#### Inspect the VPC subnets to get the AZ "a" subnet and VPC Id

The following code assumes the basic VPC setup in Studio Workshop which should have a public subnet where a we can deploy an RDS instance with a public IP attached. 

In [ ]:
print('Subnet_ID\tCidrBlock')
print('-----------------------------')
sn_all = ec2_client.describe_subnets()

for sn in sn_all['Subnets'] :
    if sn['AvailabilityZone'] == AZ:
        print(sn['SubnetId'], end=' | ')
        print(sn['CidrBlock'] )
        SUBNET = sn
        VPCID = sn['VpcId']
        break
        
print("ok")

#### Create a security group for the RDS instance with an ingress rule allowing the postgres port from our studio IP

In [ ]:
SG = ec2_client.create_security_group(GroupName='studio-rds-sg',
                                         Description='studio-rds sg',
                                         VpcId=VPCID)

print("New Security Group ID: ", SG['GroupId'])

ingress_cidr = ip_address + '/32'

print("Authorizing ingress rule for IP: ", ingress_cidr)
INGRESS = ec2_client.authorize_security_group_ingress(
            GroupId=SG['GroupId'],
            IpPermissions=[{
                'FromPort': RDS_POSTGRES_PORT,
                'IpProtocol': 'tcp',
                'IpRanges': [{
                    'CidrIp': ingress_cidr,
                    'Description': 'sagemaker studio igw ip'
                }],
                'ToPort': RDS_POSTGRES_PORT
            }],
            DryRun=False,
            TagSpecifications=[{
                'ResourceType': 'security-group-rule',
                'Tags': [{
                    'Key': 'Name',
                    'Value': 'sm-studio-rds-rule'
                }]
            }]
        )

print("ok")


### Deploy a new RDS Postgres instance and export the settings

In [ ]:
# Setup the RDS instance

RDS_RESPONSE = rds_client.create_db_instance(
                    AllocatedStorage=50,
                    DBName=DBNAME,
                    DBInstanceIdentifier=DBINSTANCE_ID,
                    DBInstanceClass=DBINSTANCE_TYPE,
                    Engine=DBENGINE,
                    EngineVersion=DBENGINE_VERSION,
                    MasterUsername=DBUSERNAME,
                    MasterUserPassword=DBPASS,
                    Port=RDS_POSTGRES_PORT,
                    VpcSecurityGroupIds=[SG['GroupId']],
                    PubliclyAccessible=True,
                    MultiAZ=False,
                    AvailabilityZone=AZ
                )


# print(RDS_RESPONSE)
print("DB Identifier\t\t|\tInstance Status")
print(RDS_RESPONSE['DBInstance']['DBInstanceIdentifier'], "\t", RDS_RESPONSE['DBInstance']['DBInstanceStatus'])


#### Check deployment status

In [ ]:
instance_desc = rds_client.describe_db_instances(DBInstanceIdentifier=DBINSTANCE_ID)
# 
instance_status = instance_desc['DBInstances'][0]['DBInstanceStatus']

print("DB Identifier\t\t|\tInstance Status")
print(instance_desc['DBInstances'][0]['DBInstanceIdentifier'], "\t", instance_status)
# print("\nEndpoint: ", instance_desc['DBInstances'][0]['Endpoint']['Address'],":",instance_desc['DBInstances'][0]['Endpoint']['Port'])

if instance_status == 'available':
    endpoint_url = instance_desc['DBInstances'][0]['Endpoint']['Address']
    dbinfo = {
        "host": endpoint_url,
        "port": RDS_POSTGRES_PORT,
        "username": DBUSERNAME,
        "password": DBPASS,
        "database": DBNAME
    }
    print("\nconnection info:\n", dbinfo)
    with open('dbsettings.json', 'w', encoding='utf-8') as f:
        json.dump(dbinfo, f, ensure_ascii=False, indent=4)
    print("dbsettings file created")

#### Test the Connection

In [ ]:
# Test the connection w/ psycopg2 client

with open('dbsettings.json', 'r', encoding='utf-8') as f:
    content = f.read()
    dbsettings = json.loads(content)

dbconn = psycopg2.connect(
            host=dbsettings['host'], 
            user=dbsettings['username'], 
            password=dbsettings['password'], 
            database=dbsettings['database'], # "postgres", 
            port=dbsettings['port'], 
            connect_timeout=10
        )
dbconn.set_session(autocommit=True)
print("connection established")
dbcur = dbconn.cursor()

print("\nThis completes this section of the lab.")